In [1]:
!pip install segmentation_models_pytorch
!pip install -q -U albumentations
!pip install timm
!pip install adamp
!pip install neptune-client
# !pip install git+https://github.com/zhanghang1989/PyTorch-Encoding.git

     |████████████████████████████████| 71kB 6.3MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 245kB 17.7MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp37-none-any.whl size=60963 sha256=d798b8728675da454ea703c2edf6c274cd1e3d33ca0f608cba8ae8e1fc4eb82f
  Stored in directory: /root/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-cp37-none-any.whl size=12420 sha256=da95d549d87a11dac0697c3c75d40c3e6bcd3b49de0619beeb567bbfbb0d76ea
  Stored in directory: /root/.cache/pip/wheels/42/1e/a9/2a578ba9ad04e776e80bf0f70d8a7f4c29ec0718b92d8f6ccd
Successfully built pretrainedmodels efficientnet-pytorch
     |████████████████████████████████| 81kB 6.9MB/s 
     |████████████████████████████████| 37.6MB 136kB/s 
     |████████████████████████████████| 952kB 56.5MB/s 
  Created wheel for adamp: filename=adamp-0.3.0-c

In [1]:
!nvidia-smi

Sun May  2 15:45:36 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import segmentation_models_pytorch as smp
from tqdm import tqdm
import gc
from adamp import AdamP

import math
from torch.optim.optimizer import Optimizer, required

# from fastai.vision.all import *

from sklearn.model_selection import GroupKFold, KFold
import torch
from torch import nn
import torchvision
import cv2
import os
import numpy as np
import pandas as pd

from torchvision import transforms
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.cuda.amp import autocast, GradScaler
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau, CosineAnnealingWarmRestarts, _LRScheduler
from scipy.ndimage.interpolation import zoom
import albumentations as A
from torch.nn import functional as F
from albumentations.pytorch import ToTensorV2

from pycocotools.coco import COCO

import matplotlib.pyplot as plt
import sys
import time
import random
import timm

import neptune
# import encoding

In [3]:
# run = neptune.init('vvvic313/trash-segmentation', api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJlODg2NjVlNC01YjIxLTQ3ZGItYWVkYS05MGFiYWNjMGI2YjUifQ==')

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [5]:
CFG = {
    "img_size": 512,
    "num_workers": 4,
    "scheduler": "Warmup",
    "epochs": 20,
    "criterion": "CELoss",
    "decoder": "DeepLabV3Plus",
    "encoder": "resnext50_32x4d",
    "pretrained": "imagenet",
    "lr": 1e-4,
    "batch_size": 8,
    "weight_decay": 1e-6,
    "gradient_accumulation_steps": 4,
    "seed": 42,
    "optimizer": "Adam",
    "mean": (0.485, 0.456, 0.406),
    "std": (0.229, 0.224, 0.225),
    "mix_prob": 0.5,
    "pseudo_label": True
}

In [6]:
seed_everything(CFG['seed'])

In [7]:
def get_train_augmentations():
    return A.Compose([
        A.HorizontalFlip(p = 0.5),
        A.VerticalFlip(p = 0.5),
        A.ShiftScaleRotate(p=0.5),
        A.Cutout(),
        A.RandomBrightnessContrast(),
        A.Normalize(mean=CFG['mean'], std=CFG['std'], max_pixel_value=255.0, p=1.0),
        ToTensorV2()
    ], p=1.0)

In [8]:
# neptune.create_experiment(name="day_0502", params=CFG)
# neptune.append_tag("effb3", "deeplabv3+",  "heavy_aug", "fold2")

In [9]:
def get_validation_augmentations():
    return A.Compose([
        A.Normalize(mean=CFG['mean'], std=CFG['std'], max_pixel_value=255.0, p=1.0),
        ToTensorV2()
    ],p=1.0)

In [10]:
class TrashDataset(Dataset):
    def __init__(self, df, root="/content/drive/MyDrive/trash_segmentation/data/", mode="train", transform=None):
        self.df = df.reset_index(drop=True).copy()
        self.mode = mode
        self.transform = transform
        self.root = root
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        image_path = self.root + self.df.iloc[idx]['filepath']
        imgs = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        
        
        if self.mode=="train" or self.mode=="val":
            mask_path = self.root + self.df.iloc[idx]['masks']
            masks = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE).astype(np.float32)
            transformed = self.transform(image=imgs, mask=masks)
            imgs = transformed["image"]
            masks = transformed["mask"]

            return imgs, masks
        
        elif self.mode == "test":
            transformed = self.transform(image=imgs)
            imgs = transformed["image"]
            
            return imgs, 1

In [11]:
df = pd.read_csv("/content/drive/MyDrive/trash_segmentation/data/train.csv")

if CFG['psuedo_label']:
    additional_df = pd.read_csv("/content/drive/MyDrive/trash_segmentation/data/test.csv")
    df = pd.concat([df, additional_df], ignore_index=True)

In [12]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [13]:
def prepare_dataloader(df, fold):
    train_ids = df[~df.Folds.isin(fold)]
    val_ids = df[df.Folds.isin(fold)]

    train_ds = TrashDataset(train_ids, mode="train", transform=get_train_augmentations())
    val_ds = TrashDataset(val_ids, mode="val", transform=get_validation_augmentations())
    
    train_loader = DataLoader(train_ds, 
                              batch_size=CFG["batch_size"], 
                              shuffle=True, 
                              num_workers=CFG["num_workers"],
                              collate_fn=collate_fn)
    val_loader = DataLoader(val_ds,
                            batch_size=CFG["batch_size"],
                            shuffle=False,
                            num_workers=CFG["num_workers"],
                            collate_fn=collate_fn)
    
    return train_loader, val_loader

In [14]:
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1e-7):
        
        inputs = inputs.log_softmax(dim=1).exp()
        
        bs = targets.size(0)
        num_classes = inputs.size(1)
        dims = (0, 2)
        
        
        targets = targets.view(bs, -1)
        inputs = inputs.view(bs, num_classes, -1)
        
        targets = F.one_hot(targets, num_classes)
        targets = targets.permute(0, 2, 1)
        
        intersection = torch.sum(inputs * targets, dim=dims)
        cardinality = torch.sum(inputs + targets, dim=dims)
        
        dice = (2.0 * intersection + smooth) / (cardinality + smooth)
        
        loss = 1 - dice
        
        return loss

In [15]:
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()
        
    def forward(self, inputs, targets, smooth=1e-7):
        
#         ce_loss = nn.CrossEntropy()(inputs, targets)
        ce_loss = F.cross_entropy(inputs, targets)
        
        inputs = inputs.log_softmax(dim=1).exp()
        
        bs = targets.size(0)
        num_classes = inputs.size(1)
        dims = (0, 2)
        
        targets = targets.view(bs, -1)
        inputs = inputs.view(bs, num_classes, -1)
        
        targets = F.one_hot(targets, num_classes)
        targets = targets.permute(0, 2, 1)
        
        intersection = torch.sum(inputs * targets, dim=dims)
        cardinality = torch.sum(inputs + targets, dim=dims)
        
        dice = (2.0 * intersection + smooth) / (cardinality + smooth)
        
        loss = 1 - dice

        mask = targets.sum(dims) > 0
        loss *= mask.to(loss.dtype)


        return (loss.mean()) +  ce_loss

In [16]:
criterion = None
if CFG['criterion'] == "DiceLoss":
    criterion = DiceLoss()
elif CFG['criterion'] == "CELoss":
    criterion = nn.CrossEntropyLoss()
elif CFG['criterion'] == "CustomLoss":
    criterion = CustomLoss()

In [17]:
class CustomCosineAnnealingWarmUpRestarts(_LRScheduler):
    def __init__(self, optimizer, T_0, T_mult=1, eta_max=0.1, T_up=0, gamma=1., last_epoch=-1):
        if T_0 <= 0 or not isinstance(T_0, int):
            raise ValueError("Expected positive integer T_0, but got {}".format(T_0))
        if T_mult < 1 or not isinstance(T_mult, int):
            raise ValueError("Expected integer T_mult >= 1, but got {}".format(T_mult))
        if T_up < 0 or not isinstance(T_up, int):
            raise ValueError("Expected positive integer T_up, but got {}".format(T_up))
        self.T_0 = T_0
        self.T_mult = T_mult
        self.base_eta_max = eta_max
        self.eta_max = eta_max
        self.T_up = T_up
        self.T_i = T_0
        self.gamma = gamma
        self.cycle = 0
        self.T_cur = last_epoch
        super(CustomCosineAnnealingWarmUpRestarts, self).__init__(optimizer, last_epoch)
        
    
    def get_lr(self):
        if self.T_cur == -1:
            return self.base_lrs
        elif self.T_cur < self.T_up:
            return [(self.eta_max - base_lr)*self.T_cur / self.T_up + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.eta_max - base_lr) * (1 + math.cos(math.pi * (self.T_cur-self.T_up) / (self.T_i - self.T_up))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.T_cur = self.T_cur + 1
            if self.T_cur >= self.T_i:
                self.cycle += 1
                self.T_cur = self.T_cur - self.T_i
                self.T_i = (self.T_i - self.T_up) * self.T_mult + self.T_up
        else:
            if epoch >= self.T_0:
                if self.T_mult == 1:
                    self.T_cur = epoch % self.T_0
                    self.cycle = epoch // self.T_0
                else:
                    n = int(math.log((epoch / self.T_0 * (self.T_mult - 1) + 1), self.T_mult))
                    self.cycle = n
                    self.T_cur = epoch - self.T_0 * (self.T_mult ** n - 1) / (self.T_mult - 1)
                    self.T_i = self.T_0 * self.T_mult ** (n)
            else:
                self.T_i = self.T_0
                self.T_cur = epoch
                
        self.eta_max = self.base_eta_max * (self.gamma**self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

In [18]:
def _fast_hist(label_true, label_pred, n_class):
    mask = (label_true >= 0) & (label_true < n_class)
    hist = np.bincount(
        n_class * label_true[mask].astype(int) +
        label_pred[mask], minlength=n_class ** 2).reshape(n_class, n_class)
    return hist


def label_accuracy_score_(label_trues, label_preds, n_class):
    """Returns accuracy score evaluation result.
      - overall accuracy
      - mean accuracy
      - mean IU
      - fwavacc
    """
    hist = np.zeros((n_class, n_class))
    for lt, lp in zip(label_trues, label_preds):
        hist += _fast_hist(lt.flatten(), lp.flatten(), n_class)
    acc = np.diag(hist).sum() / hist.sum()
    with np.errstate(divide='ignore', invalid='ignore'):
        acc_cls = np.diag(hist) / hist.sum(axis=1)
    acc_cls = np.nanmean(acc_cls)
    with np.errstate(divide='ignore', invalid='ignore'):
        iu = np.diag(hist) / (
            hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist)
        )
    mean_iu = np.nanmean(iu)
    freq = hist.sum(axis=1) / hist.sum()
    fwavacc = (freq[freq > 0] * iu[freq > 0]).sum()
    return acc, acc_cls, mean_iu, fwavacc

In [19]:
def label_accuracy_score(hist):
    """
    Returns accuracy score evaluation result.
      - [acc]: overall accuracy
      - [acc_cls]: mean accuracy
      - [mean_iu]: mean IU
      - [fwavacc]: fwavacc
    """
    acc = np.diag(hist).sum() / hist.sum()
    with np.errstate(divide='ignore', invalid='ignore'):
        acc_cls = np.diag(hist) / hist.sum(axis=1)
    acc_cls = np.nanmean(acc_cls)

    with np.errstate(divide='ignore', invalid='ignore'):
        iu = np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))
    mean_iu = np.nanmean(iu)

    freq = hist.sum(axis=1) / hist.sum()
    fwavacc = (freq[freq > 0] * iu[freq > 0]).sum()
    return acc, acc_cls, mean_iu, fwavacc

In [20]:
def add_hist(hist, label_trues, label_preds, n_class):
    """
        stack hist(confusion matrix)
    """

    for lt, lp in zip(label_trues, label_preds):
        hist += _fast_hist(lt.flatten(), lp.flatten(), n_class)

    return hist

In [21]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1.0 - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    return bbx1, bby1, bbx2, bby2

In [22]:
def cutmix(data, target, alpha):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_target = target[indices]

    lam = np.clip(np.random.beta(alpha, alpha), 0.3, 0.4)
    bbx1, bby1, bbx2, bby2 = rand_bbox(data.size(), lam)
    new_data = data.clone()
    new_target = target.clone()
    new_data[:, :, bby1:bby2, bbx1:bbx2] = data[indices, :, bby1:bby2, bbx1:bbx2]
    new_target[:, bby1:bby2, bbx1:bbx2] = target[indices, bby1:bby2, bbx1:bbx2]
    # adjust lambda to exactly match pixel ratio
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (data.size()[-1] * data.size()[-2]))
    

    return new_data, new_target

In [23]:
def train_one_epoch(epoch, model, device, optimizer, criterion, train_loader, scheduler):
    model.train()
    running_loss = None
    
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), position=0, leave=True)

    for step, (imgs, masks) in pbar:

        if (step+1) == (len(train_loader)):
            continue
        mix_decision = np.random.rand()
        imgs = torch.stack(imgs)
        masks = torch.stack(masks)
        imgs = imgs.to(device).float()
        masks = masks.to(device).long()
        
        if mix_decision < CFG['mix_prob']:
            imgs, masks = cutmix(imgs, masks, 1.0)

        with autocast():
            model.to(device)
            mask_preds = model(imgs)
            loss = criterion(mask_preds, masks) / CFG['gradient_accumulation_steps']
            scaler.scale(loss).backward()

            # loss.backward()

            if running_loss is None:
                running_loss = loss.item() * CFG['gradient_accumulation_steps']
            else:
                running_loss = running_loss * 0.99 + loss.item() * CFG['gradient_accumulation_steps'] * 0.01

            if ((step + 1) % CFG["gradient_accumulation_steps"]==0) or ((step+1) == (len(train_loader))):
                scaler.step(optimizer)
                scaler.update()
                # optimizer.zero_grad()
                # optimizer.step()                            
                optimizer.zero_grad() 
                description = f"epoch {epoch} loss: {running_loss: .4f}"
                pbar.set_description(description)
                
    scheduler.step()        

In [24]:
def valid_one_epoch(epoch, model, device, criterion, val_loader):
    model.eval()
    
    total_loss = 0
    running_loss = None
    cnt = 0
    mIoU_list = []
    pbar = tqdm(enumerate(val_loader), total=len(val_loader), position=0, leave=True)
    hist = np.zeros((12, 12))
    for step, (imgs, masks) in pbar:
        if (step+1) == (len(train_loader)):
            continue
        imgs = torch.stack(imgs)
        masks = torch.stack(masks)
        imgs = imgs.to(device).float()
        masks = masks.to(device).long()
        
        cnt += 1

        mask_preds = model(imgs)
        # print(f"{mask_preds.shape}       ")
        loss = criterion(mask_preds, masks)

        mask_preds = torch.argmax(mask_preds, dim=1).detach().cpu().numpy()
        # print(mask_preds.shape)

        mIoU = label_accuracy_score_(masks.detach().cpu().numpy(), mask_preds, n_class=12)[2]
        mIoU_list.append(mIoU)

        total_loss += loss.item()
            
        if running_loss is None:
            running_loss = loss.item()
        else:
            running_loss = running_loss * 0.99 + loss.item() * 0.01

        description = f'epoch {epoch} Loss: {running_loss:.4f}, mIoU: {np.mean(mIoU_list):.4f}'
        pbar.set_description(description)

    return total_loss/cnt, np.mean(mIoU_list)

In [25]:
FOLDS = 5
kf = KFold(FOLDS, shuffle=True, random_state=CFG['seed'])
df["Folds"] = 0

for fold, (train_idx, val_idx) in enumerate(kf.split(df)):
    df.loc[val_idx, 'Folds'] = fold

In [26]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [27]:
def create_folder(directory):
    try:
        os.makedirs(directory)
    except:
        pass

In [28]:
for fold in range(FOLDS):
    # if fold != 0:
    #   continue
    print(f"{fold} fold start")
    
    if CFG['decoder'] == "Unetpp":
        model = smp.UnetPlusPlus(CFG['encoder'], encoder_weights=CFG['pretrained'], in_channels=3, classes=12).to(device)
    elif CFG['decoder'] == 'DeepLabV3Plus':
        model = smp.DeepLabV3Plus(CFG['encoder'], encoder_weights=CFG['pretrained'], in_channels=3, classes=12).to(device)
    elif CFG['decoder'] == 'DeepLabV3':
        model = smp.DeepLabV3(CFG['encoder'], encoder_weights=CFG['pretrained'], in_channels=3, classes=12).to(device)
    elif CFG['decoder'] == "UperNet":
        model = encoding.models.sseg.UperNet(12, CFG['encoder'], aux=False)
    
    if CFG['scheduler'] == "Warmup":
        if CFG['optimizer'] == "Adam":
            optimizer = torch.optim.Adam(model.parameters(), lr=0, weight_decay=CFG['weight_decay'])
            scheduler = CustomCosineAnnealingWarmUpRestarts(optimizer, T_0=CFG['epochs'], T_mult=1, eta_max=CFG['lr'], T_up=CFG['epochs']//10, gamma=1.)
    else:
        if CFG['optimizer'] == "Adam":
            optimizer = torch.optim.Adam(model.parameters(), lr=CFG['lr'], weight_decay=CFG['weight_decay'])
    
    scaler = GradScaler()
    train_loader, valid_loader = prepare_dataloader(df, [fold])
    
    best_mIoU = 0
    num_epochs = CFG['epochs']
    
    for epoch in range(num_epochs):
        train_one_epoch(epoch, model, device, optimizer, criterion, train_loader, scheduler)

        with torch.no_grad():
            epoch_loss, mIoU = valid_one_epoch(epoch, model, device, criterion, valid_loader)

        # neptune.log_metric(f"fold{fold} epoch loss", epoch_loss)
        # neptune.log_metric(f"fold{fold} mIoU", mIoU)

        if best_mIoU < mIoU:
            best_mIoU = mIoU
            dir_ = f"/content/drive/MyDrive/trash_segmentation/models"
            create_folder(dir_)
            torch.save({'model': model.state_dict(),
                        'optimizer': optimizer.state_dict(),
                        'scheduler': scheduler.state_dict()
                        }, f"{dir_}/{CFG['encoder']}_{fold}.pth")
            print("model is saved")
        print("")

    # neptune.log_metric(f"fold {fold} Best mIoU", best_mIoU)
    del model, optimizer, train_loader, valid_loader, scheduler
    gc.collect()
    torch.cuda.empty_cache()
# neptune.stop()

0 fold start


Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/se_resnext101_32x4d-3b2fe3d8.pth" to /root/.cache/torch/hub/checkpoints/se_resnext101_32x4d-3b2fe3d8.pth


epoch 0 Loss: 2.5256, mIoU: 0.0101: 100%|██████████| 82/82 [01:39<00:00,  1.21s/it]


model is saved



epoch 1 Loss: 0.9083, mIoU: 0.2437: 100%|██████████| 82/82 [01:38<00:00,  1.20s/it]


...


epoch 16 Loss: 0.3097, mIoU: 0.4679: 100%|██████████| 82/82 [01:36<00:00,  1.18s/it]


epoch 17 Loss: 0.3079, mIoU: 0.4654: 100%|██████████| 82/82 [01:37<00:00,  1.19s/it]


epoch 18 Loss: 0.2993, mIoU: 0.4668: 100%|██████████| 82/82 [01:37<00:00,  1.19s/it]


epoch 19 Loss: 0.3033, mIoU: 0.4736: 100%|██████████| 82/82 [01:36<00:00,  1.18s/it]


model is saved

